<a href="https://colab.research.google.com/github/rajaraman61/stock-prediction/blob/main/stock_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Setup kaggle - colab
!pip install -q kaggle
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d yasserh/walmart-dataset
!unzip walmart-dataset.zip

In [ ]:
df = pd.read_csv("Walmart.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe().style.background_gradient(cmap = 'GnBu')

Highest weekly sales is **3818686.450000**

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Day'] = df['Date'].dt.weekday
df['Week'] = df['Date'].dt.week
df['Month'] = df['Date'].dt.month
df['Year']  = df['Date'].dt.year
df.drop('Date',axis=1,inplace=True)
df.head()

The date is discected into day, Month and year for further analysis.

In [ ]:
df.shape

In [ ]:
for i in df.columns:
    print(f'{i}: {df[i].nunique()}')

In [ ]:
df.columns

# EDA

In [ ]:
plt.pie(df['Year'].value_counts().values,labels =df['Year'].value_counts().index,
       autopct='%1.2f%%',shadow=True,colors=['gold','red',"silver"])
plt.title('Annual Sales')
plt.show()

Year 2011 has highest sales followed by 2010 and 2012.

In [ ]:
plt.figure(figsize=(16,8))

df.groupby('Store')['Weekly_Sales'].sum().plot(kind='bar')
plt.title('Year-Wise Sales')

plt.show()

Store number 4 and 20 have highest weekly sales.

In [ ]:
plt.figure(figsize=(8,4))

df.groupby(['Day'])['Weekly_Sales'].sum().plot(kind='line',color='Red')
plt.title('Daywise Sales')

plt.show()  

Thursday has maximum sales in a week.

In [ ]:
plt.figure(figsize=(18,6))

df.groupby(['Week'])['Weekly_Sales'].sum().plot(kind='bar', color='orange')
plt.title('Week-wise Sales')

plt.show()

41st week has highest sale in a year

In [ ]:
plt.figure(figsize=(8,4))

df.groupby('Month')['Weekly_Sales'].sum().plot(kind='bar',color='pink')
plt.title('Month-wise Sales')

plt.show()

April and December has highest sales

In [ ]:
  plt.figure(figsize=(14,10))

df.groupby('Year')['Weekly_Sales'].sum().plot(kind='bar', color="red")
plt.title('Year-Wise Sales')

plt.show()

In [ ]:
# finding outlier
n = 1
cols =["Weekly_Sales","Temperature","Fuel_Price","CPI","Unemployment"]
plt.figure(figsize=(15,10))
for i in cols:
        if n<=5:
            plt.subplot(3,2,n);
            n+=1
            sns.boxplot(x = df[i])
            plt.xlabel(i)

In [ ]:
l = ["Weekly_Sales","Temperature","Unemployment"]
def outlier_removal(l):
    for i in l:
        Q1  = df[i].quantile(0.25)
        Q3  = df[i].quantile(0.75)
        IQR = Q3-Q1
        Uper = Q3+1.5*IQR
        lower = Q1-1.5*IQR
        df[i]  = np.where(df[i]>Uper,Uper,(np.where(df[i]<lower,lower,df[i])))
outlier_removal(l)

In [ ]:
n = 1
cols =["Weekly_Sales","Temperature","Unemployment"]
plt.figure(figsize=(15,3))
for i in cols:
        if n<=3:
            plt.subplot(1,3,n);
            n+=1
            sns.boxplot(x = df[i])
            plt.xlabel(i)

In [ ]:
df2 = df.copy()
df2.head()

In [ ]:
days = {0:'Sunday',1:'Monday',2:'Tuesday',3:'Wednesday',4:'Thursday',5: 'Friday',6:'Saturday'}
df2['Day'] = df2['Day'].map(days)
months={1:'January',2:'February',3:'March',4:'April',5:'May',6:'June',7:'July',8:'August',9:'September',10:'October',11:'Novemenber',12:'December'}
df2['Month']= df2['Month'].map(months)
df2.head()

In [ ]:
plt.figure(figsize=(16,8))
plt.subplot(1, 3, 2)
plt.pie(df2['Day'].value_counts().values, labels =df2['Day'].value_counts().index,
autopct = '%1.2f%%', shadow=True,colors=["Orange",'tomato', 'cornflowerblue', 'gold', 'orchid', 'green',"#77BFE2"]) 
plt.title('Day-wise Distribution')
plt.grid()

plt.subplot(1, 3, 1)
plt.pie(df2['Month'].value_counts().values, labels =df2['Month'].value_counts().index,
autopct = '%1.2f%%',startangle=90, shadow=True,colors=sns.color_palette('Set2'))
plt.title('Month-wise Distribution')
plt.grid()

plt.subplot(1, 3, 3)
df3 = df2.groupby('Holiday_Flag')['Weekly_Sales'].sum().reset_index()
plt.pie(df2['Year'].value_counts().values, labels =df2['Year'].value_counts().index,
autopct = '%1.2f%%',startangle=90, shadow=True,colors=sns.color_palette('Set2'),labeldistance=1.1)  
plt.title('Year-wise Distribution')
plt.grid()

plt.show()

In [ ]:
plt.figure(figsize=(12,10))

sns.barplot(df2,x= 'Week',y='Weekly_Sales')
plt.title('Weekly Sales')

plt.show()

47th and 51st week has maximum weekly sales. Week 4th has minimum sales.

In [ ]:
plt.figure(figsize=(20,14))
monthly_sales = pd.pivot_table(df2, index = "Month", columns = "Year", values = "Weekly_Sales")
monthly_sales.plot()
plt.title('Yearly Sales')
plt.show()

In [ ]:
print('Minimum Sales in the Walmart: ',df2.groupby('Store')['Weekly_Sales'].sum().min())
print('Maximum Sales in the Walmart: ',df2.groupby('Store')['Weekly_Sales'].sum().max())

In [ ]:
df3 = df2.groupby('Holiday_Flag')['Weekly_Sales'].sum().reset_index()
plt.pie(df3['Weekly_Sales'],labels= ['Normal Week','Special Holiday Week'],
autopct='%1.2f%%',startangle=90,explode=[0,0.2],shadow=True,colors=['orange','brown'])
plt.show()

In [ ]:
from numpy import mean
t = 1
plt.figure(figsize=(18,16))
for i in ["Weekly_Sales","Temperature","Fuel_Price","CPI","Unemployment"]:
        if t<=5:
            plt.subplot(3,2,t)
            ax = sns.barplot(data = df2 , x = "Holiday_Flag" ,y = i  , hue = df.Holiday_Flag ,estimator=mean);
            t+=1
            
            for i in ax.containers:     #to set a label on top of the bars.
                ax.bar_label(i,)

In [ ]:
  n = 1
plt.figure(figsize=(18,16))
for i in ["Weekly_Sales","Temperature","Fuel_Price","CPI","Unemployment"]:
        if n<=5:
            plt.subplot(5,1,n)
            ax = sns.lineplot(data = df2 , x = "Year" ,y = i  , hue = df.Holiday_Flag );
            plt.xticks(rotation = 90)
            n+=1

In [ ]:
sns.pairplot(df2 , hue = "Holiday_Flag" );
plt.title("Distribution and relation of all attributes on Holiday and Normal Week");

In [ ]:
# Heatmap - to analyze the coorelation
plt.figure(figsize = (20,10))
sns.heatmap(df2.corr(),annot=True,cmap ="YlGnBu")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
df2.Holiday_Flag.value_counts()

In [ ]:
n = 1
plt.figure(figsize=(15,10))
for i in ['Weekly_Sales','Temperature', 'Fuel_Price','CPI', 'Unemployment','Year']:
        if n<=6:
            plt.subplot(3,2,n);
            n+=1
            sns.kdeplot(x = df2[i])
            plt.xlabel(i)